In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import DataLoader
from torchsummary import summary
from torchmetrics import Accuracy 

import pandas as pd 
import numpy as np
import os 

from tqdm.auto import trange

## Preporcessing

In [3]:
PATH = r"C:\Users\suyash\Desktop\toxic\tox21_dense_train.csv"
data = pd.read_csv(PATH).values
labels = pd.read_csv(PATH).columns
labels

Index(['Unnamed: 0', 'AW', 'AWeight', 'Arto', 'BertzCT', 'Chi0', 'Chi1',
       'Chi10', 'Chi2', 'Chi3',
       ...
       'W3D', 'W3DH', 'WNSA1', 'WNSA2', 'WNSA3', 'WPSA1', 'WPSA2', 'WPSA3',
       'grav', 'rygr'],
      dtype='object', length=802)

In [4]:
for x in range(799):
    if max(data[:, 3:][:, x])!=min(data[:, 3:][:, x]):
        xtr = (data[:, 3:][:, x]-np.mean(data[:, 3:][:, x]))/(max(data[:, 3:][:, x])-min(data[:, 3:][:, x]))
        data[:, 3:][:, x] = xtr
    else:
        xtr = (data[:, 3:][:, x])/(max(data[:, 3:][:, x])+min(data[:, 3:][:, x])+1e-10)+0.5
        data[:, 3:][:, x] = xtr
data = data[:, 3:]

In [5]:
BATCH_SIZE = 100
VALID_SIZE = 100

xtrain = np.array(data[:10000], dtype="float32")
xvalid = np.array(data[10000:12000], dtype="float32")

xtrain = np.array([x for x in DataLoader(torch.from_numpy(xtrain), batch_size=BATCH_SIZE)], dtype="object")
xvalid = np.array([x for x in DataLoader(torch.from_numpy(xvalid), batch_size=VALID_SIZE)], dtype="object")

C:\Users\suyash\AppData\Local\Temp/ipykernel_29528/986131156.py:7: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  xtrain = np.array([x for x in DataLoader(torch.from_numpy(xtrain), batch_size=BATCH_SIZE)], dtype="object")
C:\Users\suyash\AppData\Local\Temp/ipykernel_29528/986131156.py:8: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To r

## Try later
delaring parameters like this is not possible because i thik the __init__ function declared layers are used to determine the number of parameters


In [6]:
# class DenseBlock(nn.Module):
#     def __init__(self, inp, out):
#         super(DenseBlock, self).__init__()
#         self.dense = nn.Linear(inp, out)
#         self.batchnorm = nn.BatchNorm1d(out, momentum=0.9)
#         self.relu = nn.ReLU()

#     def forward(self, x):
#         return self.relu(self.batchnorm(self.dense(x)))

# class Autoencoder(nn.Module):
#     def __init__(self, inpdim, ndim:list):
#         super(Autoencoder, self).__init__()
#         ndim.insert(0, inpdim)
#         self.block = []

#         for n in range(len(ndim)-1):
#             self.block.append(DenseBlock(ndim[n], ndim[n+1]))
#         self.block.append(DenseBlock(ndim[-1], 1))

#         ndim.reverse()
#         self.block.append(DenseBlock(1, ndim[0]))
#         for n in range(len(ndim)-1):
#             self.block.append(DenseBlock(ndim[n], ndim[n+1]))

#     def forward(self, x):
#         for b in self.block:
#             x = b(x)
#         return x

# model = Autoencoder(799, [300,100,30,10,3])

## Model

In [11]:
class DenseBlock(nn.Module):
    def __init__(self, inp, out):
        super().__init__()
        self.dense = nn.Linear(inp, out)
        self.batchnorm = nn.BatchNorm1d(out, momentum=0.9)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.batchnorm(self.dense(x)))

class Autoencoder(nn.Module):
    def __init__(self, inp) -> None:
        super(Autoencoder, self).__init__()
        self.den1 = DenseBlock(inp, 300)
        self.den2 = DenseBlock(300, 100)
        self.den3 = DenseBlock(100, 30)
        self.den4 = DenseBlock(30, 10)
        self.den5 = DenseBlock(10, 3)
        self.den6 = DenseBlock(3, 1)
        self.den7 = DenseBlock(1, 3)
        self.den8 = DenseBlock(3, 10)
        self.den9 = DenseBlock(10, 30)
        self.den10 = DenseBlock(30, 100)
        self.den11 = DenseBlock(100, 300)
        self.den12 = DenseBlock(300, inp)

    def forward(self, x):
        return self.den12(self.den11(self.den10(self.den9(self.den8(self.den7(self.den6(self.den5(self.den4(self.den3(self.den2(self.den1(x))))))))))))

model = Autoencoder(799)

## Training

In [12]:
steps = len(xtrain)
vstep = len(xvalid)
epochs = 100
criterian = nn.MSELoss()
learning_rate = 1e-3
decay = 1
accuracy = Accuracy()
for epoch in range(epochs):
    vls = 0
    lss = 0
    learning_rate = learning_rate/(1+epoch*decay)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for step in range(steps):
        xtr = xtrain[step]

        xpred = model(xtr)
        optimizer.zero_grad()
        loss = criterian(xpred, xtr)
        lss = lss+loss
        loss.backward()
        optimizer.step()
        # acc = accuracy(xpred, xtr)

    for step in range(vstep):
        vtr = xvalid[step]

        xpred = model(vtr)
        vloss = criterian(xpred, vtr)
        vls = vls + vloss
        # vac = accuracy(xpred, vtr)

    print(f"Epoch: {epoch+1}/{epochs} || Loss: {lss/steps} || Validation Loss: {vls/vstep}\n")

Epoch: 1/100 || Loss: 0.06823854148387909 || Validation Loss: 0.015857389196753502

Epoch: 2/100 || Loss: 0.008190597407519817 || Validation Loss: 0.009160177782177925

Epoch: 3/100 || Loss: 0.007698947563767433 || Validation Loss: 0.02889552339911461

Epoch: 4/100 || Loss: 0.010018164291977882 || Validation Loss: 0.008611990138888359

Epoch: 5/100 || Loss: 0.007197219878435135 || Validation Loss: 0.022047845646739006

Epoch: 6/100 || Loss: 0.007167369592934847 || Validation Loss: 0.01650538481771946

Epoch: 7/100 || Loss: 0.007158719934523106 || Validation Loss: 0.015683161094784737

Epoch: 8/100 || Loss: 0.007157500367611647 || Validation Loss: 0.0156710185110569

Epoch: 9/100 || Loss: 0.00715735275298357 || Validation Loss: 0.015669669955968857

Epoch: 10/100 || Loss: 0.0071573443710803986 || Validation Loss: 0.015669722110033035

Epoch: 11/100 || Loss: 0.007157343439757824 || Validation Loss: 0.015669725835323334

Epoch: 12/100 || Loss: 0.007157343439757824 || Validation Loss: 0.01

KeyboardInterrupt: 